# Hello and welcome!
Here is a link to the Kaggle for convenience: https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/overview

Here is a link to the data: https://drive.google.com/drive/folders/1TgD14QwrcnvYTQKxg2xDVbDUdWAg_G3T?usp=sharing

Feature information can be found either in the `feature_info.md` file, or at the following link: https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/data

# EDA
Let's get better acquainted with our dataset.

In [1]:
import pandas as pd

In [2]:
def print_df_missing_values(df: pd.DataFrame):
    has_missing_value_col = df.isna().any()
    has_missing_value = has_missing_value_col.any()
    
    if not has_missing_value:
        print('No missing values')
    else:
        for i, col_has_missing_value in enumerate(has_missing_value_col):
            if col_has_missing_value:
                col_name = has_missing_value_col.index[i]

                n_missing = df[col_name].isna().sum()
                total = df[col_name].shape[0]

                print(f'(# Missing {col_name} / total): {n_missing}/{total}')

def print_df_info(df: pd.DataFrame):
    print(f'size: {df.shape}')
    print(f'features:\n{df.columns}')
    print_df_missing_values(df)

Our data is time sensitive:
<ul>
    <li>
        Client data: Up to 2 days before prediction date.
    </li>
    <li>
        Gas/Electricity prices: Up to 2 days before prediction date.
    </li>
    <li>
        Weather Forecast: Updates at midnight. We can use same day data for this feature.
    </li>
</ul>

Suggestion: Create lag features for our samples to get Gas/Electricity price from 2 days ago?

Our data is split into several different files. We will have to merge them for training.

We have missing target values... We should impute these rows.

In [3]:
train_df = pd.read_csv('./data/train.csv')
print_df_info(train_df)
train_df.head(4)

size: (2018352, 9)
features:
Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id'],
      dtype='object')
(# Missing target / total): 528/2018352


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1


In [4]:
gas_prices_df = pd.read_csv('./data/gas_prices.csv')
print_df_info(gas_prices_df)
gas_prices_df.head(4)

size: (637, 5)
features:
Index(['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh',
       'origin_date', 'data_block_id'],
      dtype='object')
No missing values


,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1
1,2021-09-02,45.62,46.29,2021-09-01,2
2,2021-09-03,45.85,46.40,2021-09-02,3
3,2021-09-04,46.30,46.80,2021-09-03,4


In [5]:
client_df = pd.read_csv('./data/client.csv')
print_df_info(client_df)
client_df.head(4)

size: (41919, 7)
features:
Index(['product_type', 'county', 'eic_count', 'installed_capacity',
       'is_business', 'date', 'data_block_id'],
      dtype='object')
No missing values


,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2
1,2,0,17,166.40,0,2021-09-01,2
2,3,0,688,7207.88,0,2021-09-01,2
3,0,0,5,400.00,1,2021-09-01,2


In [6]:
electricity_prices_df = pd.read_csv('./data/electricity_prices.csv')
print_df_info(electricity_prices_df)
electricity_prices_df.head(4)

size: (15286, 4)
features:
Index(['forecast_date', 'euros_per_mwh', 'origin_date', 'data_block_id'], dtype='object')
No missing values


,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1


In [7]:
forecast_weather_df = pd.read_csv('./data/forecast_weather.csv')
print_df_info(forecast_weather_df)
forecast_weather_df.head(4)

size: (3424512, 18)
features:
Index(['latitude', 'longitude', 'origin_datetime', 'hours_ahead',
       'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component',
       '10_metre_v_wind_component', 'data_block_id', 'forecast_datetime',
       'direct_solar_radiation', 'surface_solar_radiation_downwards',
       'snowfall', 'total_precipitation'],
      dtype='object')
(# Missing surface_solar_radiation_downwards / total): 2/3424512


,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,57.6,21.7,2021-08-31 23:00:00+00:00,1,15.655786,11.553613,0.904816,0.019714,0.000000,0.905899,-0.411328,-9.106137,1,2021-09-01 00:00:00+00:00,0.0,0.0,0.0,0.0
1,57.6,22.2,2021-08-31 23:00:00+00:00,1,13.003931,10.689844,0.886322,0.004456,0.000000,0.886658,0.206347,-5.355405,1,2021-09-01 00:00:00+00:00,0.0,0.0,0.0,0.0
2,57.6,22.7,2021-08-31 23:00:00+00:00,1,14.206567,11.671777,0.729034,0.005615,0.000000,0.730499,1.451587,-7.417905,1,2021-09-01 00:00:00+00:00,0.0,0.0,0.0,0.0
3,57.6,23.2,2021-08-31 23:00:00+00:00,1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,2021-09-01 00:00:00+00:00,0.0,0.0,0.0,0.0


In [8]:
historical_weather_df = pd.read_csv('./data/historical_weather.csv')
print_df_info(historical_weather_df)
historical_weather_df.head(4)

size: (1710802, 18)
features:
Index(['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall',
       'surface_pressure', 'cloudcover_total', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_high', 'windspeed_10m',
       'winddirection_10m', 'shortwave_radiation', 'direct_solar_radiation',
       'diffuse_radiation', 'latitude', 'longitude', 'data_block_id'],
      dtype='object')
No missing values


,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-01 00:00:00,14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.083333,8,0.0,0.0,0.0,57.6,21.7,1.0
1,2021-09-01 00:00:00,13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.111111,359,0.0,0.0,0.0,57.6,22.2,1.0
2,2021-09-01 00:00:00,14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.333333,355,0.0,0.0,0.0,57.6,22.7,1.0
3,2021-09-01 00:00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.083333,297,358.0,277.0,81.0,57.6,23.2,1.0


In [9]:
weather_station_to_county_df = pd.read_csv('./data/weather_station_to_county_mapping.csv')
print_df_info(weather_station_to_county_df)
weather_station_to_county_df.head(4)

size: (112, 4)
features:
Index(['county_name', 'longitude', 'latitude', 'county'], dtype='object')
(# Missing county_name / total): 63/112
(# Missing county / total): 63/112


,county_name,longitude,latitude,county
0,NaN,21.7,57.6,NaN
1,NaN,21.7,57.9,NaN
2,NaN,21.7,58.2,NaN
3,NaN,21.7,58.5,NaN
